In [1]:
#Notebook to visualize the macro function (which is responsible to detect if the trend is globally bear or bull)
import sys
sys.path.append("..")

from memory_profiler import profile
from core import indicators as ic
from core import strat
from core.common import VBTfunc
import vectorbtpro as vbt
from vectorbtpro.utils.config import Config
import numpy as np
import gc
import talib
import pandas as pd
import math
import copy
from numba import njit

from core.macro import VBTMACROTREND, VBTMACROVIS

In [2]:
class Vis(VBTfunc):
    def __init__(self,period,**kwargs):
        
        self.close_dic={}
        self.open_dic={}
        self.low_dic={}
        self.high_dic={}
        self.indexes=["CAC40", "DAX", "NASDAQ"]
        
        for ind in self.indexes:
            super().__init__(ind,period)
            
            if kwargs.get("index",False):
            #self.only_index=True
                self.close_dic[ind]=self.close_ind
                self.open_dic[ind]=self.open_ind
                self.low_dic[ind]=self.low_ind
                self.high_dic[ind]=self.high_ind
            else:
                self.close_dic[ind]=self.close
                self.open_dic[ind]=self.open
                self.low_dic[ind]=self.low
                self.high_dic[ind]=self.high        
        
        self.macro_trend={}

        for ind in self.indexes:
            t=VBTMACROTREND.run(self.close_dic[ind])
            self.macro_trend[ind]=t.macro_trend

    def macro_mode(self,mode_to_vis):
        self.ents={}
        self.exs={}
        
        for ind in self.indexes:
            t=VBTMACROVIS.run(self.macro_trend[ind],mode_to_vis=mode_to_vis)
            self.ents[ind]=t.entries
            self.exs[ind]=t.exits
    
    def manual_calculate_pf(self,ind, mode_to_vis):
        self.macro_mode(mode_to_vis)
        pf=vbt.Portfolio.from_signals(self.close_dic[ind], self.ents[ind],self.exs[ind],
                                      freq="1d")
        return pf
        
      

In [3]:
o=Vis("2007_2022") #,index=True

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/69 [00:00<?, ?it/s]

In [8]:
pf=o.manual_calculate_pf("NASDAQ",0)
pf.returns_stats()

/home/maxime/.local/lib/python3.8/site-packages/vectorbtpro/portfolio/base.py:9537: UserWarning:

Object has multiple columns. Aggregated some metrics using <function mean at 0x7f0b241731f0>. Pass column to select a single column/group.



Start                           2007-01-03 00:00:00+00:00
End                             2021-12-31 00:00:00+00:00
Period                                 3777 days 00:00:00
Total Return [%]                               121.643145
Benchmark Return [%]                           1761.29449
Annualized Return [%]                            3.966245
Annualized Volatility [%]                       21.240637
Max Drawdown [%]                                44.589527
Max Drawdown Duration        2021 days 20:52:10.434782624
Sharpe Ratio                                      0.37389
Calmar Ratio                                     0.121704
Omega Ratio                                      1.121022
Sortino Ratio                                    0.568953
Skew                                             0.330878
Kurtosis                                        52.671447
Tail Ratio                                       1.140656
Common Sense Ratio                               1.188551
Value at Risk 

In [9]:
pf=vbt.Portfolio.from_signals(o.close_dic["CAC40"], o.ents["CAC40"],o.exs["CAC40"], freq="1d")

In [10]:
#pf.plot()
pf.plot(column=pf.wrapper.columns[3])

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'na…